In [ ]:
#import some tools that we will use
import json
import skimage.io
import skimage.measure
import skimage.feature
import skimage.segmentation
from skimage.morphology import disk
import scipy.ndimage
import numpy
import matplotlib.pyplot as plt
from generate_spot_positions import GenerateSpotPositionsJob
from models.image_filename_glob import ImageFilenameGlob
from models.generate_spot_positions_config import GenerateSpotPositionsConfig
import csv
import scipy.ndimage
from pathlib import Path

In [ ]:
#define paths to images
image_folder = "C:\\Users\\finne\\Documents\\python\\cell_crops_20210304\\"

well_ID = "B02"
field_ID = 1
channel_ID = 3

# Pick out 10 nuclei (glob says it is random but it's almost always the first ten)
image_path = Path(image_folder)

image_filename_glob = image_path.glob(str(ImageFilenameGlob(well = well_ID, f = field_ID, c = channel_ID, suffix="_maximum_projection_nuclear_mask_???", extension="npy")))

images_selected_random = [next(image_filename_glob) for _ in range(10)]

for image_selected in images_selected_random:
    print(image_selected)

In [ ]:
# SET PARAMETERS HERE! 

# Contrast threshold is applied to the laplacian of the gaussian, it is fairly cryptic, so trial and error works best to find a good value.
local_contrast_threshold = 2

# Peak radius is applied to pre-filter the spots; anything smaller than this will get filtered out, so err on the small side. 
# It ought to be near the actual radius of the visible spot in pixels, though.
peak_radius = 4

# Because your images are normalized to 0-1 range in the cell, your spot maxima are set to 1. So this ought to be the inverse of your overall signal:noise ratio
# Counterintuitively, better FISH -> lower threshold here -> less filtering taking place on the actual image
global_contrast_threshold = 0.4

segmenters = [GenerateSpotPositionsJob(image, "no_output", local_contrast_threshold, peak_radius, global_contrast_threshold) for image in images_selected_random]

In [ ]:
# display image and found spots -- these coordinates (+ offset) can be used to look up precalculated z-position and radial position. 

fig, axs = plt.subplots(2,5)

for i in range(0,10):
    ax = axs[i//5, i%5]
    ax.imshow(segmenters[i].image)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    for spot in segmenters[i].spots:
        circle=plt.Circle((spot[1], spot[0]), 7, color='r', alpha=0.3)
        ax.add_patch(circle)

fig.set_size_inches(15,7)
fig.canvas.draw()

In [ ]:
# read in config file into hash
# change hash as needed to reflect parameters above
# overwrite config file with new values

config_file = "C:\\Users\\finne\\Documents\\python\\dropbox_test\\spot_seg_config.json"

config_file_path = Path(config_file)

existing_generate_spot_positions_configs = []
if config_file_path.exists():
    with open(config_file_path) as json_file:
        existing_generate_spot_positions_configs_json_params = json.load(json_file)
        existing_generate_spot_positions_configs = [
            GenerateSpotPositionsConfig.from_json_params(json_params)
            for json_params
            in existing_generate_spot_positions_configs_json_params
        ]

current_generate_spot_positions_config = GenerateSpotPositionsConfig(
    channel=channel_ID,
    local_contrast_threshold=local_contrast_threshold,
    peak_radius=peak_radius,
    global_contrast_threshold=global_contrast_threshold
)

updated_generate_spot_positions_configs = [
    current_generate_spot_positions_config,
    *(
        existing_generate_spot_positions_config
        for existing_generate_spot_positions_config
        in existing_generate_spot_positions_configs
        if existing_generate_spot_positions_config.channel != current_generate_spot_positions_config.channel
    )
]

updated_generate_spot_positions_configs_json_params = [config.to_json_params() for config in updated_generate_spot_positions_configs]
with open(config_file_path, 'w') as write_file:
    json.dump(updated_generate_spot_positions_configs_json_params, write_file)
